In [1]:

import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

class SensorNeuron(nn.Module):
    def __init__(self, input_size, dataset_samples, threshold=0.5):
        super(SensorNeuron, self).__init__()
        self.input_size = input_size
        self.dataset_samples = dataset_samples
        self.threshold = threshold  # Limiar para sinal forte
        self.sensory_db = self.initialize_sensory_neurons()  # Banco de dados sensorial
        self.consolidation_counter = [0] * input_size  # Contadores de consolidação
    
    def initialize_sensory_neurons(self):
        sensory_neurons = []
        for _ in range(self.input_size):
            random_sample = random.choice(self.dataset_samples)
            sensory_neurons.append(random.choice(random_sample))  # Neurônios inicializados com dados aleatórios
        return torch.tensor(sensory_neurons).float()
    
    def forward(self, x):
        x = x.float()  # Garantir tipo correto de entrada
        reinforcement_score = 0  # Inicia o score de reforço
        for i in range(self.input_size):
            # Calcula a diferença absoluta
            diff = torch.abs(x[:, i] - self.sensory_db[i])
            
            # Verifica se cada elemento no lote é menor que o threshold
            signal_strong = diff < self.threshold
            
            # Verifica se a maioria dos sinais é forte (True)
            if signal_strong.all():  # Se todos os valores no lote são menores que o limiar
                self.consolidation_counter[i] += 1
                reinforcement_score += 1  # Sinal forte encontrado, reforça a informação
            else:
                self.consolidation_counter[i] -= 1  # Sinal fraco, reduz a consolidação
            
            # Caso o contador de consolidação seja negativo, atualiza o neurônio
            if self.consolidation_counter[i] < 0:
                self.sensory_db = self.update_sensory_neuron(self.sensory_db, self.dataset_samples, i)  # Chama a função externa
        return x, reinforcement_score
    
    def update_sensory_neuron(self, sensory_db, dataset_samples, index):
        random_sample = random.choice(dataset_samples)
        new_value = random.choice(random_sample)
        sensory_db[index] = new_value
        return sensory_db
        
    def reinforce_database(self):
        # Caso o sinal seja encontrado mais de uma vez, a informação é consolidada no banco de dados permanente
        for i in range(self.input_size):
            if self.consolidation_counter[i] > 5:  # Exemplo de limiar de consolidação
                self.sensory_db[i] = self.sensory_db[i]  # Reforço da informação
            elif self.consolidation_counter[i] < -3:
                self.update_sensory_neuron(i)  # Atualiza os neurônios com base no contador de consolidação

class AdvancedMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dataset_samples, threshold=0.5):
        super(AdvancedMLP, self).__init__()
        self.sensor_neuron = SensorNeuron(input_size=input_size, dataset_samples=dataset_samples, threshold=threshold)  # Neurônios sensoriais
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x, reinforcement_score = self.sensor_neuron(x)  # Processamento inicial com neurônios sensoriais
        x = F.relu(self.fc1(x))  # Primeira camada de rede neural
        x = self.fc2(x)  # Camada final de saída
        return F.log_softmax(x, dim=1), reinforcement_score



In [ ]:
# Carregar o dataset Iris
iris = load_iris()
X = iris.data  # Características (features)
y = iris.target  # Rótulos (labels)

# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Converter para tensores do PyTorch
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Criar DataLoader para treino
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Criar DataLoader para teste
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Definir o modelo
input_size = X_train.shape[1]  # 4 características de entrada
hidden_size = 16
output_size = len(set(y))  # 3 classes (Setosa, Versicolor, Virginica)
dataset_samples = [list(range(10))]  # Exemplo simples, substitua conforme necessário
threshold = 0.5

model = AdvancedMLP(input_size=input_size, hidden_size=hidden_size, output_size=output_size, dataset_samples=dataset_samples, threshold=threshold)

# Definir a função de perda e o otimizador
criterion = nn.CrossEntropyLoss()  # Para classificação
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Treinamento do modelo
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_preds = 0
    total_samples = 0

    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()
        
        # Forward pass
        outputs, reinforcement_score = model(inputs)
        
        # Calcular a perda
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Calcular acurácia
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * correct_preds / total_samples
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Avaliar o modelo nos dados de teste
model.eval()
correct_preds = 0
total_samples = 0

with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch
        outputs, _ = model(inputs)
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_samples += labels.size(0)

test_accuracy = 100 * correct_preds / total_samples
print(f"Test Accuracy: {test_accuracy:.2f}%")


In [2]:
# Carregar o dataset Iris
digits = load_digits()
X = digits.data  # Características (features)
y = digits.target  # Rótulos (labels)

# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Converter para tensores do PyTorch
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Criar DataLoader para treino
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Criar DataLoader para teste
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Definir o modelo
input_size = X_train.shape[1]  # 4 características de entrada
hidden_size = 16
output_size = len(set(y))  # 3 classes (Setosa, Versicolor, Virginica)
dataset_samples = [list(range(10))]  # Exemplo simples, substitua conforme necessário
threshold = 0.5

model = AdvancedMLP(input_size=input_size, hidden_size=hidden_size, output_size=output_size, dataset_samples=dataset_samples, threshold=threshold)

# Definir a função de perda e o otimizador
criterion = nn.CrossEntropyLoss()  # Para classificação
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Treinamento do modelo
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_preds = 0
    total_samples = 0

    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()
        
        # Forward pass
        outputs, reinforcement_score = model(inputs)
        
        # Calcular a perda
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Calcular acurácia
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * correct_preds / total_samples
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Avaliar o modelo nos dados de teste
model.eval()
correct_preds = 0
total_samples = 0

with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch
        outputs, _ = model(inputs)
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_samples += labels.size(0)

test_accuracy = 100 * correct_preds / total_samples
print(f"Test Accuracy: {test_accuracy:.2f}%")


Epoch [1/100], Loss: 2.1019, Accuracy: 41.27%
Epoch [2/100], Loss: 1.6669, Accuracy: 67.64%
Epoch [3/100], Loss: 1.2214, Accuracy: 75.99%
Epoch [4/100], Loss: 0.8730, Accuracy: 83.79%
Epoch [5/100], Loss: 0.6506, Accuracy: 88.10%
Epoch [6/100], Loss: 0.5087, Accuracy: 89.56%
Epoch [7/100], Loss: 0.4156, Accuracy: 91.16%
Epoch [8/100], Loss: 0.3501, Accuracy: 92.41%
Epoch [9/100], Loss: 0.3021, Accuracy: 93.18%
Epoch [10/100], Loss: 0.2647, Accuracy: 94.08%
Epoch [11/100], Loss: 0.2360, Accuracy: 94.78%
Epoch [12/100], Loss: 0.2124, Accuracy: 95.34%
Epoch [13/100], Loss: 0.1923, Accuracy: 95.55%
Epoch [14/100], Loss: 0.1761, Accuracy: 96.10%
Epoch [15/100], Loss: 0.1615, Accuracy: 96.66%
Epoch [16/100], Loss: 0.1495, Accuracy: 97.08%
Epoch [17/100], Loss: 0.1388, Accuracy: 97.15%
Epoch [18/100], Loss: 0.1289, Accuracy: 97.49%
Epoch [19/100], Loss: 0.1205, Accuracy: 97.84%
Epoch [20/100], Loss: 0.1131, Accuracy: 98.05%
Epoch [21/100], Loss: 0.1064, Accuracy: 97.98%
Epoch [22/100], Loss: 